In [1]:
!pip install psycopg2
import psycopg2 as ps
import pandas as pd

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [34]:
# upload your CSV files here
df_tracks = pd.read_csv('doggo.csv', index_col=0)
df_artists = pd.read_csv('artists.csv', index_col=0)
df_playlists = pd.read_csv('playlists.csv', index_col=0)
df_playlists

,playlist_id,playlist_name,avg_energy,avg_loudness,avg_tempo,avg_valence,avg_danceability,top_genres,total_tracks,total_duration
0,7pdD7G1UnufPRNcw3dE072,K,0.7830,-5.4331,127.9779,0.5667,0.5643,"['rock', 'punk', 'pop punk', 'socal pop punk',...",934,55:48:31
1,5EJJgEGKlNOFyPKDMzW9W6,떡볶이,0.8104,-3.4164,123.5307,0.6388,0.7148,"['k-pop', 'k-pop girl group', 'korean ost', 'k...",141,07:48:44
2,1CC50bbyEzbK4XNqLeRUo2,b,0.6436,-6.5986,116.7140,0.5595,0.6949,"['pop', 'boy band', 'hip hop', 'rap', 'dance p...",34,02:02:58
3,6h7zqwUfM8SlLdFAGOWUbs,never forget,0.6270,-8.7333,119.8720,0.6369,0.6205,"['soft rock', 'mellow gold', 'new wave pop', '...",76,05:05:06
4,77xt0OncByfy5OLkvNWlox,whatever simulation,0.6549,-6.2083,116.8082,0.5566,0.6937,"['k-pop', 'korean r&b', 'pop', 'k-rap', 'r&b']",77,04:37:58
5,0ur6GzNfyA6kAIqFKC5Zlr,untitled,0.7516,-6.9592,122.7545,0.6250,0.5568,"['rock', 'pop rock', 'classic rock', 'alternat...",53,03:44:07
6,7D7XMbpAoyhWD8LFIuwqSS,i feel like,0.8537,-4.5121,137.9943,0.5474,0.4894,"['pop punk', 'rock', 'modern rock', 'emo', 'pr...",115,06:47:44
7,1g5zPlCoUV6dZfj5wdMgNy,not edgy,0.8456,-5.2929,133.1573,0.5184,0.4677,"['skate punk', 'punk', 'alternative metal', 'r...",328,19:19:45
8,1zPYYyIjusJdmAFMEuMCc3,doggo,0.7510,-5.7541,127.2553,0.5495,0.5197,"['modern rock', 'rock', 'alternative rock', 'i...",99,05:50:46


In [33]:
def connect_to_db(host_name, dbname, port, username, password):
  
  try:
    conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)
  except ps.OperationalError as e:
    raise e
  else:
    print('Connected!')
  
  return conn


def create_table_tracks(curr):
  create_table_command = ("""CREATE TABLE IF NOT EXISTS table_name (
                      track_id VARCHAR(255) PRIMARY KEY,
                      track_title TEXT NOT NULL,
                      track_artists TEXT NOT NULL,
                      track_album TEXT NOT NULL,
                      track_release_date TEXT NOT NULL,
                      energy FLOAT NOT NULL,
                      loudness FLOAT NOT NULL,
                      tempo FLOAT NOT NULL,
                      valence FLOAT NOT NULL,
                      danceability FLOAT NOT NULL,
                      duration TEXT NOT NULL
              )""")
  
  curr.execute(create_table_command)

def create_table_artists(curr):
  create_table_command = ("""CREATE TABLE IF NOT EXISTS artists (
                      artist_id VARCHAR(255) PRIMARY KEY,
                      artist_name TEXT NOT NULL,
                      genres TEXT NOT NULL,
                      popularity INTEGER NOT NULL
              )""")
  
  curr.execute(create_table_command)


def create_table_playlists(curr):
  create_table_command = ("""CREATE TABLE IF NOT EXISTS playlists (
                      playlist_id VARCHAR(255) PRIMARY KEY,
                      playlist_name TEXT NOT NULL,
                      avg_energy FLOAT NOT NULL,
                      avg_loudness FLOAT NOT NULL,
                      avg_tempo FLOAT NOT NULL,
                      avg_valence FLOAT NOT NULL,
                      avg_danceability FLOAT NOT NULL,
                      top_genres TEXT NOT NULL,
                      total_tracks INTEGER NOT NULL,
                      total_duration TEXT NOT NULL
              )""")

  curr.execute(create_table_command)


def insert_into_table_tracks(curr, track_id, track_title, track_artists, track_album, track_release_date,
                             energy, loudness, tempo, valence, danceability, duration):
  insert_into_tracks = ("""INSERT INTO table_name (track_id, track_title, track_artists, track_album, track_release_date,
  energy, loudness, tempo, valence, danceability, duration)
  VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);""")
  row_to_insert = (track_id, track_title, track_artists, track_album, track_release_date, energy, loudness, tempo, valence, danceability, duration)
  curr.execute(insert_into_tracks, row_to_insert)

  
def insert_into_table_artists(curr, artist_id, artist_name, genres, popularity):
  insert_into_artists = ("""INSERT INTO artists (artist_id, artist_name, genres, popularity)
  VALUES(%s,%s,%s,%s);""")
  row_to_insert = (artist_id, artist_name, genres, popularity)
  curr.execute(insert_into_artists, row_to_insert)


def insert_into_table_playlists(curr, playlist_id, playlist_name, avg_energy, avg_loudness, avg_tempo, avg_valence, avg_danceability, 
                                top_genres, total_tracks, total_duration):
  insert_into_playlists = ("""INSERT INTO playlists (playlist_id, playlist_name, avg_energy, avg_loudness,
  avg_tempo, avg_valence, avg_danceability, top_genres, total_tracks, total_duration)
  VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);""")
  row_to_insert = (playlist_id, playlist_name, avg_energy, avg_loudness, avg_tempo, avg_valence, avg_danceability,
                   top_genres, total_tracks, total_duration)
  curr.execute(insert_into_playlists, row_to_insert)


def check_if_track_exists(curr, track_id): 
  query = ("""SELECT track_id FROM table_name WHERE track_id = %s""")
  curr.execute(query, (track_id,))

  return curr.fetchone() is not None


def check_if_artist_exists(curr, artist_id): 
  query = ("""SELECT artist_id FROM artists WHERE artist_id = %s""")
  curr.execute(query, (artist_id,))

  return curr.fetchone() is not None


def check_if_playlist_exists(curr, playlist_id): 
  query = ("""SELECT playlist_id FROM playlists WHERE playlist_id = %s""")
  curr.execute(query, (playlist_id,))

  return curr.fetchone() is not None


def append_from_df_to_db_tracks(curr, df_tracks):
  for i, row in df_tracks.iterrows():
      insert_into_table_tracks(curr, row['track_id'], row['track_title'], row['track_artists'], row['track_album'],
                               row['track_release_date'], row['energy'], row['loudness'], row['tempo'],
                               row['valence'], row['danceability'], row['duration'])


def append_from_df_to_db_artists(curr, df_artists):
  for i, row in df_artists.iterrows():
      insert_into_table_artists(curr, row['artist_id'], row['artist_name'], row['genres'], row['popularity'])


def append_from_df_to_db_playlists(curr, df_playlists):
  for i, row in df_playlists.iterrows():
      insert_into_table_playlists(curr, row['playlist_id'], row['playlist_name'], row['avg_energy'], row['avg_loudness'], row['avg_tempo'],
                               row['avg_valence'], row['avg_danceability'], row['top_genres'], row['total_tracks'], row['total_duration'])


def update_db_tracks(curr, df_tracks):
  tmp_df = pd.DataFrame(columns=['track_id', 'track_title', 'track_artists', 'track_album',
                                 'track_release_date', 'energy', 'loudness', 'tempo',
                                 'valence', 'danceability', 'duration'])
  for i, row in df_tracks.iterrows():
      if check_if_track_exists(curr, row['track_id']):
        continue
      else:  # The track doesn't exist so we will add it to a temp df and append it using append_from_df_to_db
          tmp_df = tmp_df.append(row)

  return tmp_df


def update_db_artists(curr, df_artists):
  tmp_df = pd.DataFrame(columns=['artist_id', 'artist_name', 'genres', 'popularity'])
  for i, row in df_artists.iterrows():
      if check_if_artist_exists(curr, row['artist_id']):
        continue
      else:
          tmp_df = tmp_df.append(row)

  return tmp_df


def update_db_playlists(curr, df_playlists):
  tmp_df = pd.DataFrame(columns=['playlist_id', 'playlist_name', 'avg_energy', 'avg_loudness', 'avg_tempo',
                                 'avg_valence', 'avg_danceability', 'top_genres', 'total_tracks', 'total_duration'])
  for i, row in df_playlists.iterrows():
      if check_if_playlist_exists(curr, row['playlist_id']):
        continue
      else:
          tmp_df = tmp_df.append(row)

  return tmp_df

In [31]:
# main

# database credentials
host_name = '******.rds.amazonaws.com'
dbname = '******'
port = '5432'
username = '******' 
password = '******'
conn = None

# establish connection to db
conn = connect_to_db(host_name, dbname, port, username, password)
curr = conn.cursor()

Connected!


In [32]:
# create table
create_table_tracks(curr)
create_table_artists(curr)
create_table_playlists(curr)

# update db for new items
new_track_df = update_db_tracks(curr, df_tracks)
new_artist_df = update_db_artists(curr, df_artists)
new_playlist_df = update_db_playlists(curr, df_playlists)
conn.commit()

# insert new items into db table
append_from_df_to_db_tracks(curr, new_track_df)
append_from_df_to_db_artists(curr, new_artist_df)
append_from_df_to_db_playlists(curr, new_playlist_df)
conn.commit()